In [31]:
import os
import json
import openai
from langchain.tools import tool
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
from typing import List, Dict, Annotated
from math import radians, sin, cos, sqrt, atan2

# ✅ 환경 변수 로드
load_dotenv()

# ✅ OpenAI API 키 설정
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(api_key=OPENAI_API_KEY)

# ✅ 목업 병원 데이터 (테스트용)
MOCK_HOSPITALS = [
    {"name": "서울 중앙 병원", "address": "서울시 강남구 테헤란로 123", "rating": 4.8, "latitude": 37.498, "longitude": 127.027},
    {"name": "한빛 정형외과", "address": "서울시 서초구 반포대로 45", "rating": 4.2, "latitude": 37.491, "longitude": 127.015},
    {"name": "힐링 치과", "address": "서울시 강서구 강서로 89", "rating": 4.5, "latitude": 37.540, "longitude": 126.837},
    {"name": "사랑 요양 병원", "address": "서울시 송파구 올림픽대로 78", "rating": 3.9, "latitude": 37.519, "longitude": 127.114},
    {"name": "서울대학교 병원", "address": "서울시 종로구 대학로 103", "rating": 4.7, "latitude": 37.579, "longitude": 127.002},
]

# ✅ 기본 위치 데이터 (서울 중심)
DEFAULT_LATITUDE = 37.5665
DEFAULT_LONGITUDE = 126.9780
# 사용자 위치 정보에서, 반경(3km로 검색)

# ✅ 반경 기준 (3km)
RADIUS_KM = 3

def haversine(lat1, lon1, lat2, lon2):
    """
    두 위도/경도 좌표 간의 실제 거리(km)를 계산하는 하버사인 공식 적용.
    """
    R = 6371  # 지구 반지름 (단위: km)
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)

    a = sin(dlat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c  # 두 지점 간의 거리(km)

@tool
def search_nearby_hospitals(
    latitude: Annotated[float, "User's latitude (위도)"] = None,
    longitude: Annotated[float, "User's longitude (경도)"] = None
) -> List[Dict[str, str]]:
    """
    사용자의 위치(GPS 좌표)를 기반으로 가까운 병원을 추천하는 기능.
    """

    # ✅ 사용자가 위치 정보를 제공하지 않으면 기본값(서울 중심) 설정
    if latitude is None or longitude is None:
        latitude = 37.5665  # 서울 중심 위도
        longitude = 126.9780  # 서울 중심 경도

    # ✅ float 변환 (LangChain 오류 방지)
    latitude = float(latitude)
    longitude = float(longitude)

    # ✅ 병원 데이터 필터링 (가까운 순으로 정렬)
    nearby_hospitals = sorted(
        MOCK_HOSPITALS,
        key=lambda hospital: abs(hospital["latitude"] - latitude) + abs(hospital["longitude"] - longitude),
    )[:3]

    return [
        {
            "name": hospital["name"],
            "address": hospital["address"],
            "rating": hospital["rating"],
            "latitude": hospital["latitude"],
            "longitude": hospital["longitude"]
        }
        for hospital in nearby_hospitals
    ]

# tools 정의
tools = [search_nearby_hospitals]



# ✅ 프롬프트 템플릿 수정 (위치 정보를 포함)
prompt = ChatPromptTemplate.from_messages(
    [
        # ✅ 시스템 역할 정의 (에이전트의 행동 가이드)
        (
            "system",
            "당신은 사용자 위치를 기반으로 병원을 추천하는 챗봇입니다. "
            "Make sure to use the `search_nearby_hospitals` tool for finding hospitals near the user. "
            "User's location: latitude={latitude}, longitude={longitude}."
        ),
        # ✅ 대화 히스토리 (멀티턴 대화 지원)
        ("placeholder", "{chat_history}"),
        # ✅ 사용자의 입력
        ("human", "{input}"),
        # ✅ 에이전트가 검색하거나 추론한 내용을 기록하는 공간
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# ✅ 프롬프트 확인 (디버깅 용도)
print(prompt)

# LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Agent 생성
agent = create_tool_calling_agent(llm, tools, prompt) # llm 모델, 정의한 tool, 정의된 prompt

# AgentExecutor 생성
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=10,
    max_execution_time=10,
    handle_parsing_errors=True,
)

Python-dotenv could not parse statement starting at line 13
Python-dotenv could not parse statement starting at line 15
Python-dotenv could not parse statement starting at line 16
Python-dotenv could not parse statement starting at line 17
Python-dotenv could not parse statement starting at line 18
Python-dotenv could not parse statement starting at line 19
Python-dotenv could not parse statement starting at line 20
Python-dotenv could not parse statement starting at line 21
Python-dotenv could not parse statement starting at line 22
Python-dotenv could not parse statement starting at line 23
Python-dotenv could not parse statement starting at line 24


input_variables=['input', 'latitude', 'longitude'] optional_variables=['agent_scratchpad', 'chat_history'] input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} partial_variables={'chat_history': [], 'agent_scratchpad': []} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['latitude', 'longitude'], template="당신은 사용자 위치를 기반으로 병원을 추천하는 챗봇입니다. Make sure to use the `search_nearby_hospitals` tool for f

In [32]:
# AgentExecutor 실행
result = agent_executor.invoke({
    "input": "근처 병원을 알려주세요.",
    "latitude": DEFAULT_LATITUDE,   # 서울 중심 좌표
    "longitude": DEFAULT_LONGITUDE  # 서울 중심 좌표
})

print("Agent 실행 결과:")
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `search_nearby_hospitals` with `{'latitude': 37.5665, 'longitude': 126.978}`


[{'name': '서울대학교 병원', 'address': '서울시 종로구 대학로 103', 'rating': 4.7, 'latitude': 37.579, 'longitude': 127.002}, {'name': '한빛 정형외과', 'address': '서울시 서초구 반포대로 45', 'rating': 4.2, 'latitude': 37.491, 'longitude': 127.015}, {'name': '서울 중앙 병원', 'address': '서울시 강남구 테헤란로 123', 'rating': 4.8, 'latitude': 37.498, 'longitude': 127.027}]근처의 병원은 다음과 같습니다:

1. **서울대학교 병원**
   - 주소: 서울시 종로구 대학로 103
   - 평점: 4.7

2. **한빛 정형외과**
   - 주소: 서울시 서초구 반포대로 45
   - 평점: 4.2

3. **서울 중앙 병원**
   - 주소: 서울시 강남구 테헤란로 123
   - 평점: 4.8

필요한 정보가 더 있으면 말씀해 주세요!

> Finished chain.
Agent 실행 결과:
근처의 병원은 다음과 같습니다:

1. **서울대학교 병원**
   - 주소: 서울시 종로구 대학로 103
   - 평점: 4.7

2. **한빛 정형외과**
   - 주소: 서울시 서초구 반포대로 45
   - 평점: 4.2

3. **서울 중앙 병원**
   - 주소: 서울시 강남구 테헤란로 123
   - 평점: 4.8

필요한 정보가 더 있으면 말씀해 주세요!


In [30]:
# AgentExecutor 실행
result = agent_executor.invoke({
    "input": "내 근처 병원을 알려주세요.",
})

print("Agent 실행 결과:")
print(result["output"])



> Entering new AgentExecutor chain...


KeyError: "Input to ChatPromptTemplate is missing variables {'latitude', 'longitude'}.  Expected: ['input', 'latitude', 'longitude'] Received: ['input', 'intermediate_steps', 'agent_scratchpad']\nNote: if you intended {latitude} to be part of the string and not a variable, please escape it with double curly braces like: '{{latitude}}'."

In [26]:
# ✅ 테스트 코드: `search_nearby_hospitals()`


def search_nearby_hospitals(
    latitude: Annotated[float, "User's latitude (위도)"],
    longitude: Annotated[float, "User's longitude (경도)"]
) -> List[Dict[str, str]]:
    """
    사용자의 위치(GPS 좌표)를 기반으로 가까운 병원을 추천하는 기능.
    """

    # ✅ LangChain이 float을 처리하는 데 문제가 없도록 str 변환
    latitude = float(latitude)
    longitude = float(longitude)

    # ✅ 병원 데이터 필터링 (가까운 순으로 정렬)
    nearby_hospitals = sorted(
        MOCK_HOSPITALS,
        key=lambda hospital: abs(hospital["latitude"] - latitude) + abs(hospital["longitude"] - longitude),
    )[:3]

    return [
        {
            "name": hospital["name"],
            "address": hospital["address"],
            "rating": hospital["rating"],
            "latitude": hospital["latitude"],
            "longitude": hospital["longitude"]
        }
        for hospital in nearby_hospitals
    ]



if __name__ == "__main__":
    test_latitude = 37.498  # 강남역 근처
    test_longitude = 127.027

    print("\n🔹 [TEST] 병원 추천 툴 실행 결과:")
    result = search_nearby_hospitals(test_latitude, test_longitude)
    print(json.dumps(result, indent=4, ensure_ascii=False))



🔹 [TEST] 병원 추천 툴 실행 결과:
[
    {
        "name": "서울 중앙 병원",
        "address": "서울시 강남구 테헤란로 123",
        "rating": 4.8,
        "latitude": 37.498,
        "longitude": 127.027
    },
    {
        "name": "한빛 정형외과",
        "address": "서울시 서초구 반포대로 45",
        "rating": 4.2,
        "latitude": 37.491,
        "longitude": 127.015
    },
    {
        "name": "서울대학교 병원",
        "address": "서울시 종로구 대학로 103",
        "rating": 4.7,
        "latitude": 37.579,
        "longitude": 127.002
    }
]
